<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/machine_learning/blob/main/7_XGBoost/xgboost.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>

# XGBoost

In [11]:
!pip3 install -U scikit-learn --upgrade # necesitamos sklearn >= 0.24

In [12]:
!pip install xgboost

In [13]:
!pip install lightgbm

In [14]:
!pip install catboost

In [15]:
!pip install sweetviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 30.5 MB/s eta 0:00:00


In [16]:
!pip install ipywidgets # puede ser necesario si lo corren local

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [17]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.5 MB/s eta 0:00:00


## Introducción

Vamos a trabajar con un dataset famoso llamado **adult** tomado de UCI: https://archive.ics.uci.edu/ml/datasets/Adult . Son datos obtenidos del censo de Estados Unidos de 1994 y la tarea consiste en predecir si una persona gana más de 50.000 dólares al año o no. Por lo tanto, cada fila es una persona y las columnas son las características de la misma.

In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv("https://storage.googleapis.com/humai-datasets/aws_s3/ml_avanzado/2_Boosting_de_arboles/adult_train.csv")

In [20]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'target'],
      dtype='object')

In [21]:
df.shape

(32561, 15)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  target          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


- Las variables categóricas son: **workclass**, **education**, **marital-status**, **occupation**, **relationship**, **race**, **sex**, **native-country**.

- Las variables numéricas son: **age**, **fnlwgt**, **education-num**, **capital-gain**, **capital-loss**, **hours-per-week**

## Análisis exploratorio

Para este EDA vamos a usar un paquete que nos genera numerosos gráficos automáticamente y, por lo tanto, nos permite una aproximación rápida a los datos con los que vamos a trabajar.

In [23]:
# sweetviz es un paquete de análisis exploratorio automático, una alternativa también es pandas-profiling
import sweetviz as sv
#analyzing the dataset
advert_report = sv.analyze(df)

                                             |          | [  0%]   00:00 -> (? left)

El siguiente comando escribe un *html* con una serie de gráficos. Además, intenta abrir una pestaña nueva. Si ésto no sucede entonces deberán ejecutar manualmente el archivo. Para eso, si están en Google Colab deberán descargar el archivo y luego darle doble clic al archivo en su computadora.

In [24]:
advert_report.show_html('df.html')

Report df.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### Observaciones

Del reporte de *sweetviz* interesan destacar las siguientes cuestiones:


1- En **workclass** los nulos se identifican con **?**.

2- **target** está parejamente distribuido, con 76% para la categoría **<=50K** y 24% para **>50K**.

3- **capital-gain** y **capital-loss** son variables continuas pero mayoritariamente son 0 (91.7% y 95.3% respectivamente).

Cliqueando en ASSOCIATIONS (arriba al medio) podemos ver métricas de correlación entre variables, allí se observa que:

4- **education-num**, **capital-gain**, **hours-per-week**, **age**, **relationship** y en menor medida **marital-status** son las variables más correlacionadas con **target**.

## Preparamos train y test

## trabajamos con ordinal encoder y label encoder

In [25]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [26]:
X_train = df.drop("target",axis=1)
y_train = df["target"]

In [27]:
# LabelEncoder se usa para codificar las labels como números
l_encoder = LabelEncoder()

In [28]:
# Al hacer fit el objeto aprende un mapeo de cada label a un número que luego usaremos en test.
# Al hacer transform ejecuta el mapeo. fit_transform primero ejecuta fit y luego transform
y_train = l_encoder.fit_transform(y_train)

In [29]:
test = pd.read_csv("https://storage.googleapis.com/humai-datasets/aws_s3/ml_avanzado/2_Boosting_de_arboles/adult_test.csv")

In [30]:
X_test = test.drop("target",axis=1)
y_test = test["target"]

In [31]:
l_encoder.transform(test["target"])

array([0, 0, 1, ..., 0, 0, 1])

In [32]:
# acá estamos traduciendo las etiquetas de las labels a números, con el mismo mapeo de antes (!!!!)
y_test = l_encoder.transform(y_test)

In [33]:
# inverse_transform nos permite hacer el mapeo en el sentido opuesto, de número a label
l_encoder.inverse_transform(y_test)

array(['<=50K', '<=50K', '>50K', ..., '<=50K', '<=50K', '>50K'],
      dtype=object)

In [34]:
# Para chequear que el mapeo se hace correctamente podemos hacer lo siguiente:

In [78]:
np.array_equal(test["target"].values, l_encoder.inverse_transform(y_test))  ## controlde la tranbsformacion

True

## Entreno modelos

In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [37]:
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder

In [38]:
import scipy.stats as st

In [39]:
from xgboost.sklearn import XGBClassifier

In [40]:
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [41]:
from catboost import CatBoostClassifier

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
from sklearn.model_selection import RandomizedSearchCV

In [44]:
cat_columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]

num_columns = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

In [45]:
# vamos a usar polynomial features sobre las variables numéricas
poly = PolynomialFeatures()

In [46]:
# Las categorías "nuevas" que aparezcan se imputan con el valor de unknown_value
o_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=99999999)

Los modelos que vamos a probar son XGBoost, LightGBM, CatBoost y Regresión logística.

Armamos un pipeline en donde:

1- Usamos `ColumnTransformer` para seleccionar las columnas numéricas y aplicar `PolynomialFeatures`. `remainder="passthrough"` hace que el resto de las columnas pasen sin cambios. De este paso vamos a optimizar el hiperparámetro que indica el grado del polinomio: `selector_num__poly__degree`.

2- Luego aplicamos una transformación sobre las variables categóricas. Para los modelos basados en árboles usamos `OrdinalEncoder` que codifica las categorías como números. De este paso vamos a optimizar el hiperparámetro que indica cómo se representan las categorías nuevas, no vistas en train, con `ord_encoder__unknown_value`. Por último, para la regresión logística aplicamos one hot encoding.

3- Finalmente aplicamos un clasificador.

Nota: los nombres de los hiperparámetros que vamos a optimizar se llaman como `nombre del step + __ + nombre del hiperparámetro`

In [81]:
pipeline = Pipeline([
    ("selector_num", ColumnTransformer(  ## seleccionar las columnas numericas
        [
            ("poly", poly, num_columns)  ## columnas numericas con poly
        ], remainder="passthrough")
    ),
    ("cat_encoder", o_encoder), ## se puede aplicar sobre las variables categoricas
    ("classifier", CatBoostClassifier())
    ]
)

In [82]:
pipeline.get_params()

{'memory': None,
 'steps': [('selector_num',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('poly', PolynomialFeatures(),
                                    ['age', 'fnlwgt', 'education-num',
                                     'capital-gain', 'capital-loss',
                                     'hours-per-week'])])),
  ('cat_encoder',
   OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=99999999)),
  ('classifier', <catboost.core.CatBoostClassifier at 0x78c71e927a00>)],
 'transform_input': None,
 'verbose': False,
 'selector_num': ColumnTransformer(remainder='passthrough',
                   transformers=[('poly', PolynomialFeatures(),
                                  ['age', 'fnlwgt', 'education-num',
                                   'capital-gain', 'capital-loss',
                                   'hours-per-week'])]),
 'cat_encoder': OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=99999999),
 'classifier': <ca

In [83]:
one_to_left = st.beta(10, 1) # Esta distribución nos dará valores entre 0 y 1 mayormente cercanos a 1

In [50]:
# este diccionario de parámetros lo vamos a usar tanto para XGBoost como para LightGBM
params = {
    "classifier__n_estimators": st.randint(20,150), # n. de árboles.
    "classifier__max_depth": st.randint(3, 12),     # máxima profundiad del árbol
    "classifier__learning_rate": st.uniform(0.05, 0.4), # learning rate (xgb’s “eta”)
    "classifier__colsample_bytree": one_to_left, # ratio de la muestra de las columnas a la hora de construir el árbol
    "classifier__subsample": one_to_left,     # ratio de la muestra de las observaciones
    'classifier__reg_alpha': st.uniform(0.05,10),   # término de regularización L1
    "classifier__min_child_weight": st.uniform(1,20), # Suma mínima del peso de la instancia (hessiano) requerida en los hijos
}

In [51]:
other_params = {"selector_num__poly__degree": [1, 2, 3],
                "cat_encoder__unknown_value": [np.nan, 99999999]}

Para optimizar modelos con distintos hiperparámetros se puede usar una lista de diccionarios, en donde cada diccionario es un hiperparámetro distinto.

En primer lugar, classifier "pisa" el clasificador del pipeline original, en este caso `CatBoostClassifier`.

En segundo lugar, pasamos los parámetros que definimos en `params` (usados en XGBoost y LightGBM) y `other_params` (que son los hiperparámetro que no corresponden al clasificador).

En tercer lugar, también incluímos otros hiperparámetros de los clasificadores.

In [52]:
search_space = [{"classifier": [XGBClassifier(n_jobs=3, eval_metric="logloss", use_label_encoder=False)],
                 **params,
                 **other_params,
                "classifier__gamma": st.uniform(0, 10), # reducción mínima de la pérdida requerida para hacer un split
                },
                {"classifier": [LGBMClassifier(n_jobs=3, verbose=-1)],
                 **params,
                **other_params},
                {"classifier": [CatBoostClassifier(thread_count=3, verbose=False)],
                 **other_params,
                 "classifier__n_estimators": st.randint(20,150), # n. de árboles.
                "classifier__max_depth": st.randint(3, 12),     # máxima profundiad del árbol
                "classifier__learning_rate": st.uniform(0.05, 0.4), # learning rate (xgb’s “eta”)
                "classifier__subsample": one_to_left,     # ratio de la muestra de las observaciones
                },
               {
                   "classifier": [LogisticRegression(max_iter=1000)],
                   "classifier__C": [10**x for x in range(-5, 3)],
                   "cat_encoder": [OneHotEncoder(handle_unknown="ignore")],
               }]

In [53]:
model = RandomizedSearchCV(pipeline, search_space, n_iter=10, verbose=1, cv=3)

Comenzamos el entrenamiento

In [54]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The L

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('selector_num',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('poly',
                                                                               PolynomialFeatures(),
                                                                               ['age',
                                                                                'fnlwgt',
                                                                                'education-num',
                                                                                'capital-gain',
                                                                                'capital-loss',
                                                                                'hours-per-week'])])),
                                             ('cat_encoder',
                                              OrdinalEncoder(handle_unknown='use_encoded_value',
                                                             unknown_value=99999999)),
                                             ('classifier',
                                              <catboost.c...
                                         'classifier__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78c72cb12b00>,
                                         'classifier__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78c72dd5c4f0>,
                                         'selector_num__poly__degree': [1, 2,
                                                                        3]},
                                        {'cat_encoder': [OneHotEncoder(handle_unknown='ignore')],
                                         'classifier': [LogisticRegression(max_iter=1000)],
                                         'classifier__C': [1e-05, 0.0001, 0.001,
                                                           0.01, 0.1, 1, 10,
                                                           100]}],
                   verbose=1)

Guardamos los resultados

In [55]:
results = pd.DataFrame(model.cv_results_)

In [56]:
results.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_cat_encoder__unknown_value', 'param_classifier',
       'param_classifier__learning_rate', 'param_classifier__max_depth',
       'param_classifier__n_estimators', 'param_classifier__subsample',
       'param_selector_num__poly__degree',
       'param_classifier__colsample_bytree',
       'param_classifier__min_child_weight', 'param_classifier__reg_alpha',
       'param_cat_encoder', 'param_classifier__C', 'param_classifier__gamma',
       'params', 'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'],
      dtype='object')

In [57]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_cat_encoder__unknown_value,param_classifier,param_classifier__learning_rate,param_classifier__max_depth,param_classifier__n_estimators,param_classifier__subsample,...,param_cat_encoder,param_classifier__C,param_classifier__gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,10.683926,5.951636,3.343108,0.536782,NaN,<catboost.core.CatBoostClassifier object at 0x...,0.445790,3.0,120.0,0.960947,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': nan, 'classifie...",0.854339,0.858393,0.849166,0.853966,0.003776,8
1,7.330195,0.332027,2.987157,0.357557,99999999.0,"LGBMClassifier(n_jobs=3, verbose=-1)",0.108435,10.0,113.0,0.964399,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': 99999999, 'clas...",0.859959,0.864935,0.869345,0.864746,0.003834,2
2,8.763063,2.332758,0.855462,0.022326,NaN,LogisticRegression(max_iter=1000),NaN,NaN,NaN,NaN,...,OneHotEncoder(handle_unknown='ignore'),100.0,NaN,{'cat_encoder': OneHotEncoder(handle_unknown='...,0.859775,0.861986,0.863816,0.861859,0.001652,3
3,1.200949,0.004308,0.201197,0.006780,99999999.0,<catboost.core.CatBoostClassifier object at 0x...,0.233195,8.0,120.0,0.851171,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': 99999999, 'clas...",0.864198,0.864198,0.870082,0.866159,0.002774,1
4,1.835197,0.215017,0.968848,0.236824,99999999.0,"LGBMClassifier(n_jobs=3, verbose=-1)",0.107608,3.0,65.0,0.963896,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': 99999999, 'clas...",0.854063,0.859775,0.862803,0.858880,0.003624,5
5,3.347896,1.335528,0.904169,0.004046,NaN,<catboost.core.CatBoostClassifier object at 0x...,0.067865,4.0,95.0,0.856434,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': nan, 'classifie...",0.854432,0.856827,0.863264,0.858174,0.003729,6
6,0.597759,0.007179,0.201062,0.002425,NaN,<catboost.core.CatBoostClassifier object at 0x...,0.144398,5.0,54.0,0.959009,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': nan, 'classifie...",0.856551,0.858577,0.863816,0.859648,0.003061,4
7,0.402156,0.037757,0.170157,0.005667,NaN,"XGBClassifier(base_score=None, booster=None, c...",0.393131,8.0,68.0,0.941552,...,NaN,NaN,7.520086,"{'cat_encoder__unknown_value': nan, 'classifie...",NaN,NaN,NaN,NaN,NaN,9
8,14.935296,0.056398,3.109033,0.317751,99999999.0,<catboost.core.CatBoostClassifier object at 0x...,0.157822,8.0,119.0,0.995319,...,NaN,NaN,NaN,"{'cat_encoder__unknown_value': 99999999, 'clas...",0.850931,0.857380,0.856077,0.854796,0.002784,7
9,0.459569,0.012023,0.177134,0.006468,NaN,"XGBClassifier(base_score=None, booster=None, c...",0.119016,9.0,132.0,0.993809,...,NaN,NaN,6.157600,"{'cat_encoder__unknown_value': nan, 'classifie...",NaN,NaN,NaN,NaN,NaN,9


El mejor modelo es...

In [58]:
model.best_estimator_

Pipeline(steps=[('selector_num',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('poly',
                                                  PolynomialFeatures(degree=1),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('cat_encoder',
                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=99999999)),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x78c72d42a500>)])

Para a dibujar la curva ROC, para eso primero usamos `predict_proba` y luego importamos `plot_roc` de `scikitplot`.

In [59]:
y_predicted = model.predict_proba(X_test)

In [60]:
!pip install scikit-plot

In [61]:
!pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.


In [62]:
!pip install --upgrade scikit-learn  # Update scikit-learn
# Restart the kernel after installation



In [63]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d # Import interp1d instead of interp


In [64]:
def plot_roc(y_true, y_probas, title='ROC Curves', plot_micro=True, plot_macro=True, classes_to_plot=None, ax=None,
             figsize=None, cmap='nipy_spectral', title_fontsize="large", text_fontsize="medium"):
    """Generates the ROC curves from labels and predicted scores/probabilities
    Args:
        y_true (array-like, shape (n_samples)):
            Ground truth (correct) target values.
        y_probas (array-like, shape (n_samples, n_classes)):
            Prediction probabilities for each class returned by a classifier.
        title (string, optional): Title of the generated plot. Defaults to
            "ROC Curves".
        plot_micro (boolean, optional): Plot the micro average ROC curve.
            Defaults to ``True``.
        plot_macro (boolean, optional): Plot the macro average ROC curve.
            Defaults to ``True``.
        classes_to_plot (list-like, optional): Classes for which the ROC
            curve should be plotted. If specified, only curves for these
            classes will be plotted. Defaults to ``None``, which plots curves
            for all classes.
        ax (:class:`matplotlib.axes.Axes`, optional): The axes upon which to
            plot the curve. If None, the plot is drawn on a new set of axes.
        figsize (2-tuple, optional): Tuple denoting figure size of the plot
            e.g. (6, 6). Defaults to ``None``.
        cmap (string or :class:`matplotlib.colors.Colormap` instance, optional):
            Colormap used for plotting the projection. View Matplotlib colormaps
            for available options.
            https://matplotlib.org/users/colormaps.html
            Defaults to "nipy_spectral".
        title_fontsize (string or int, optional): Matplotlib-style fontsizes.
            Use e.g. "small", "medium", "large" or integer values. Defaults to
            "large".
        text_fontsize (string or int, optional): Matplotlib-style fontsizes.
            Use e.g. "small", "medium", "large" or integer values. Defaults to
            "medium".
    Returns:
        ax (:class:`matplotlib.axes.Axes`): The axes on which the plot was
            drawn.
    Example:
        >>> import scikitplot as skplt
        >>> nb = GaussianNB()
        >>> nb = nb.fit(X_train, y_train)
        >>> y_probas = nb.predict_proba(X_test)
        >>> skplt.metrics.plot_roc(y_test, y_probas)
        <matplotlib.axes._subplots.AxesSubplot object at 0x7fe967d64490>
        >>> plt.show()
    .. image:: _static/examples/plot_roc.png
       :align: center
       :alt: ROC Curves
    """
    y_true = np.array(y_true)
    y_probas = np.array(y_probas)



In [65]:
plot_roc(y_test, y_predicted)

In [66]:
from sklearn.metrics import roc_auc_score

In [67]:
roc_auc_score(y_test, y_predicted[:,1])

0.923470282599

## Uso CatBoost con categorical features

In [68]:
categorical_features_indices = np.where(X_train.dtypes == object)[0]

In [69]:
model = CatBoostClassifier(
    random_seed=42,
    logging_level='Silent'
)

In [70]:
from catboost import Pool, metrics, cv

cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X_train, y_train, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [71]:
# obtengo la mejor epoch
cv_data["test-Logloss-mean"].argmin()

996

In [72]:
# vamos a reentrenar con 811 steps

In [73]:
model = CatBoostClassifier(
    iterations=811,
    random_seed=42,
    logging_level='Silent',
    cat_features=categorical_features_indices
)

In [74]:
model.fit(X_train, y_train)

In [75]:
predictions_probs = model.predict_proba(X_test)

In [76]:
plot_roc(y_test, predictions_probs)

In [77]:
roc_auc_score(y_test, predictions_probs[:,1])

0.9282309716192427